In [1]:
#Dependencies
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [2]:
#prepare database file to be connected
engine=create_engine("sqlite:///hawaii.sqlite")

#Create base class
Base = automap_base()

#Be able to reflect schema of SQLite tables into code and create mappings
Base.prepare(engine, reflect=True)

In [3]:
#View Classes by Automap
Base.classes.keys()

['measurement', 'station']

In [4]:
#Give variable name to classes
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
#Create session link to database
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(engine)
session = Session()

In [6]:
#Set calculation for 2 years from 2017
prev_years=dt.date(2017, 12, 31) - dt.timedelta(days=730)

In [7]:
#Perform query to retrieve data and precipitation for JUNE
results_june=session.query(Measurement.date, Measurement.prcp).\
filter(extract('month', Measurement.date)==6).\
filter(Measurement.date >=prev_years).all()

#Save query as dataframe
df_june=pd.DataFrame(results_june, columns=['date','precipitation']).dropna()
df_june.set_index(df_june['date'], inplace=True)
df_june=df_june.sort_index()

#Summary statistic for June's precipitation
june_stats=df_june.describe()
june_stats.rename(columns={'precipitation':'June Precipitation'}, inplace=True)
june_stats

,June Precipitation
count,344.000000
mean,0.166424
std,0.281631
min,0.000000
25%,0.000000
50%,0.040000
75%,0.210000
max,1.700000


In [8]:
#Perform query to retrieve data and temperature for JUNE
results_june_temp=session.query(Measurement.date, Measurement.tobs).\
filter(extract('month', Measurement.date)==6).\
filter(Measurement.date >=prev_years).all()

#Save query as dataframe
df_june_temp=pd.DataFrame(results_june_temp, columns=['date','tobs']).dropna()
df_june_temp.set_index(df_june_temp['date'], inplace=True)
df_june_temp=df_june_temp.sort_index()

#Summary stats for June's temperature
june_temp_stats=df_june_temp.describe()
june_temp_stats.rename(columns={'tobs':'June Temp'}, inplace=True)
june_temp_stats

,June Temp
count,385.000000
mean,76.189610
std,3.215019
min,64.000000
25%,74.000000
50%,77.000000
75%,79.000000
max,83.000000


In [9]:
#Perform query to retrieve data and precipitation for DECEMBER
results_dec=session.query(Measurement.date, Measurement.prcp).\
filter(extract('month', Measurement.date)==12).\
filter(Measurement.date >=prev_years).all()

#Save query as dataframe
df_dec=pd.DataFrame(results_dec, columns=['date','precipitation']).dropna()
df_dec.set_index(df_dec['date'], inplace=True)
df_dec=df_dec.sort_index()

#Summary of data for DECEMBER
dec_stats=df_dec.describe()
dec_stats.rename(columns={'precipitation':'Dec Precipitation'}, inplace=True)
dec_stats

,Dec Precipitation
count,178.000000
mean,0.199494
std,0.342967
min,0.000000
25%,0.020000
50%,0.060000
75%,0.247500
max,2.370000


In [10]:
#Perform query to retrieve data and temperatire for DECEMBER
results_dec_temp=session.query(Measurement.date, Measurement.tobs).\
filter(extract('month', Measurement.date)==12).\
filter(Measurement.date >=prev_years).all()

#Save query as dataframe
df_dec_tobs=pd.DataFrame(results_dec_temp, columns=['date','tobs']).dropna()
df_dec_tobs.set_index(df_dec_tobs['date'], inplace=True)
df_dec_tobs=df_dec_tobs.sort_index()

#Summary of data for DECEMBER
dec_temp_stats=df_dec_tobs.describe()
dec_temp_stats.rename(columns={'tobs':'Dec Temp'}, inplace=True)
dec_temp_stats

,Dec Temp
count,200.000000
mean,71.130000
std,3.419292
min,60.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,78.000000


In [11]:
df_compare=pd.concat([june_stats, dec_stats, june_temp_stats, dec_temp_stats], axis=1)
df_compare

,June Precipitation,Dec Precipitation,June Temp,Dec Temp
count,344.000000,178.000000,385.000000,200.000000
mean,0.166424,0.199494,76.189610,71.130000
std,0.281631,0.342967,3.215019,3.419292
min,0.000000,0.000000,64.000000,60.000000
25%,0.000000,0.020000,74.000000,69.000000
50%,0.040000,0.060000,77.000000,71.000000
75%,0.210000,0.247500,79.000000,74.000000
max,1.700000,2.370000,83.000000,78.000000


In [12]:
#Find min, max, and avg temp from most stations
results_stations=session.query(Measurement.station, func.min(Measurement.tobs),\
              func.max(Measurement.tobs),\
              func.avg(Measurement.tobs)).\
group_by(Measurement.station).all()
results_stations

[('USC00511918', 53.0, 87.0, 71.61596766043456),
 ('USC00513117', 59.0, 85.0, 72.68918420081211),
 ('USC00514830', 58.0, 85.0, 74.8732970027248),
 ('USC00516128', 58.0, 84.0, 70.91500765696784),
 ('USC00517948', 58.0, 87.0, 74.68440233236151),
 ('USC00518838', 58.0, 83.0, 72.72407045009784),
 ('USC00519281', 54.0, 85.0, 71.66378066378067),
 ('USC00519397', 56.0, 87.0, 74.55323054331865),
 ('USC00519523', 56.0, 85.0, 74.5436493068565)]

In [13]:
results_stations_df=pd.DataFrame(results_stations, columns=['station','Min','Max','Avg'])
results_stations_df


,station,Min,Max,Avg
0,USC00511918,53.0,87.0,71.615968
1,USC00513117,59.0,85.0,72.689184
2,USC00514830,58.0,85.0,74.873297
3,USC00516128,58.0,84.0,70.915008
4,USC00517948,58.0,87.0,74.684402
5,USC00518838,58.0,83.0,72.724070
6,USC00519281,54.0,85.0,71.663781
7,USC00519397,56.0,87.0,74.553231
8,USC00519523,56.0,85.0,74.543649
